# Simulation template

In this notebook we run the machine learning analysis of topological phase transitions occurring  in both nearest-neighbours SSH models (ssh1) and second neighbours models (ssh2) as decribed in the paper [Machine learning topological phases in real space](https://arxiv.org/abs/1901.01963).

## Defining parameters

In [1]:
%%time
%load_ext autoreload
%autoreload 2
from simulation import *

CPU times: user 733 ms, sys: 284 ms, total: 1.02 s
Wall time: 571 ms


In [2]:
%%time
### Dataset and simulation parameters
csv_path = SSH1_PERIODIC_100_6561_CSV 
model_name = "DecisionTreeClassifier"
model_kw = {"criterion":"entropy"}
allowed_windings = [0,1]
simulation_dir = SSH1_PERIODIC_100_6561_SIMULATION_DIR
val_split = 0.9  
features_to_use = None 
shuffle_features = False
random_state = 137                    

### Running a simulation
n_experiments = 5
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
######### DON'T SET THIS TO TRUE UNLESS YOUR DATASET IS SMALL!! WILL FLOOD YOUR MEMORY!!!
store_in_lists = False   
########## BELOW ARE THE PARAMETERS THAT CONTROL WHAT WILL BE SAVED
save_eigenvector=True
save_hamiltonian=True 
save_accuracy=True 
save_models=True

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [3]:
# Parameters
model_kw = {"criterion": "entropy"}
allowed_windings = [0, 1]
val_split = 0.9
features_to_use = None
shuffle_features = False
n_experiments = 100
start_n = 0
fit_params = None
shuffle_rows = True
pred_params = None
random_features = False
store_in_lists = False
save_eigenvector = True
save_hamiltonian = True
save_accuracy = True
save_models = True
csv_path = "/home/rio/ssh_csvs/ssh1/periodic_220_6561.csv"
model_name = "DecisionTreeClassifier"
simulation_dir = "/home/rio/ssh_simulations/ssh1/periodic_220_6561"
random_state = 383


In [4]:
%%time
#Starting an instance of Simulation with a decision tree
model = DecisionTreeClassifier(criterion="entropy")
#dict_args = {"csv_path":csv_path, "model":model, "allowed_windings":allowed_windings,\
#             "simulation_dir":simulation_dir, "val_split":val_split, "features_to_use":features_to_use,\
#            "shuffle_features":shuffle_features, "random_state":random_state}
simulation = Simulation(csv_path,model_name,model_kw,allowed_windings,simulation_dir,val_split,features_to_use,\
                       shuffle_features,random_state)
#simulation = Simulation(**dict_args)

print("Info on all data: \n")
simulation.dataframe.info()
simulation.dataframe.head()

/home/rio/ml_top_phases/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Info on all data: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1443420 entries, 0 to 1443419
Columns: 228 entries, id to feat219
dtypes: float64(223), int32(3), object(2)
memory usage: 2.4+ GB
CPU times: user 25.5 s, sys: 1.11 s, total: 26.6 s
Wall time: 26.6 s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.067420,0.067420,...,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.095346,0.095307,...,0.091484,0.090680,0.092868,0.092214,0.093950,0.093447,0.094725,0.094376,0.095191,0.094997
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.095346,0.095191,...,0.080210,0.077137,0.085564,0.083022,0.089801,0.087826,0.092868,0.091484,0.094725,0.093950
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.067420,-0.067420,...,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,666,test,0.095346,-0.095307,...,0.091484,-0.090680,0.092868,-0.092214,0.093950,-0.093447,0.094725,-0.094376,0.095191,-0.094997


#### Checking initialization

In [5]:
%%time
n_features = simulation.n_features
n_hamiltonians = simulation.n_hamiltonians
n_ts = simulation.n_ts

print("n_features: ", n_features)
print("n_hamiltonians: ", n_hamiltonians)
print("n_ts: ", n_ts)

n_features:  220
n_hamiltonians:  6561
n_ts:  2
CPU times: user 629 µs, sys: 34 µs, total: 663 µs
Wall time: 397 µs


In [6]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.9358329522938577
% val:  0.0
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  6140
number of val hamiltonians:  0
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 161, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 

## Running a simulation

In [7]:
%%time
simulation.run_simulation(n_experiments, start_n, fit_params, shuffle_rows, pred_params, random_features, \
                          store_in_lists, save_eigenvector, save_hamiltonian, save_accuracy,\
                          save_models)

simulation.dataframe.head(10)

running experiments:   0%|          | 0/100 [00:00<?, ?it/s]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   1%|          | 1/100 [00:45<1:15:49, 45.96s/it]

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments:   2%|▏         | 2/100 [01:31<1:15:02, 45.94s/it]

...

THIS IS  predict_params:  None
THIS IS  the type of predict_params:  <class 'NoneType'>


running experiments: 100%|██████████| 100/100 [1:15:55<00:00, 45.23s/it]

running experiments: 100%|██████████| 100/100 [1:15:55<00:00, 45.56s/it]

CPU times: user 1h 15min 45s, sys: 11.2 s, total: 1h 15min 56s
Wall time: 1h 15min 55s


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
0,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.067420,0.067420,...,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420,0.067420
1,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.095346,0.095307,...,0.091484,0.090680,0.092868,0.092214,0.093950,0.093447,0.094725,0.094376,0.095191,0.094997
2,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.095346,0.095191,...,0.080210,0.077137,0.085564,0.083022,0.089801,0.087826,0.092868,0.091484,0.094725,0.093950
3,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.067420,-0.067420,...,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420
4,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,0.095346,-0.095307,...,0.091484,-0.090680,0.092868,-0.092214,0.093950,-0.093447,0.094725,-0.094376,0.095191,-0.094997
5,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,0.095346,-0.095191,...,0.080210,-0.077137,0.085564,-0.083022,0.089801,-0.087826,0.092868,-0.091484,0.094725,-0.093950
6,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.095346,0.085564,...,0.013569,0.029464,0.083022,-0.053818,0.087826,-0.095191,0.024239,-0.062439,-0.058223,0.018935
7,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.000142,-0.002581,...,0.026726,-0.029329,0.021458,-0.024102,0.016119,-0.018796,0.010728,-0.013429,0.005302,-0.008018
8,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,0,test,-0.000068,-0.005375,...,0.051491,-0.055988,0.042008,-0.046825,0.031977,-0.037053,0.021529,-0.026797,0.010801,-0.016191
9,0,H_-2_-2_0.501338.dat,-2.0,-2.0,0.501338,999,1,test,-0.095346,0.094725,...,-0.039608,0.029463,-0.058223,0.049236,-0.073811,0.066450,-0.085563,0.080210,-0.092868,0.089801


#### Viewing a random sample

In [8]:
%%time
simulation.dataframe.sample(frac=0.1, replace=False)

CPU times: user 253 ms, sys: 2 µs, total: 253 ms
Wall time: 252 ms


,id,path,t1,t2,winding,phase,pred_phase,type_of,feat0,feat1,...,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219
193895,881,H_-1.5_1.55_1.00055.dat,-1.50,1.55,1.000551,1,0,val,0.095342,0.074177,...,-0.091215,-0.054295,0.050038,0.089605,0.065774,0.008736,-0.083480,-0.094047,-0.023330,0.039081
1061203,4823,H_0.95_0.2_-0.000289225.dat,0.95,0.20,-0.000289,0,0,train,-0.000437,-0.007502,...,-0.094313,-0.093015,0.086548,0.083344,-0.071771,-0.066921,0.051180,0.045077,-0.026443,-0.019580
309983,1409,H_-1.15_-0.4_-0.000578451.dat,-1.15,-0.40,-0.000578,0,0,val,0.067420,-0.067420,...,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420,0.067420,-0.067420
1199662,5453,H_1.35_-0.7_0.000370348.dat,1.35,-0.70,0.000370,0,0,val,-0.095346,0.095274,...,0.080210,-0.078140,-0.085564,0.083859,0.089801,-0.088484,-0.092868,0.091956,0.094725,-0.094229
145819,662,H_-1.6_-1.3_-0.00470013.dat,-1.60,-1.30,-0.004700,0,0,val,-0.004412,-0.073081,...,-0.088470,0.025345,0.031067,0.052868,0.047781,-0.094587,-0.093647,0.071015,0.074870,0.001578
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251642,1143,H_-1.3_-1.55_1.00672.dat,-1.30,-1.55,1.006725,1,1,val,-0.092868,0.029086,...,0.018934,0.070513,-0.039608,-0.054145,0.058222,0.034962,-0.073811,-0.013962,0.085563,-0.007764
633390,2879,H_-0.25_0.2_0.000482041.dat,-0.25,0.20,0.000482,0,0,val,0.095346,0.094854,...,-0.039597,-0.030605,0.058213,0.050263,-0.073804,-0.067308,0.085558,0.080854,-0.092866,-0.090198
380662,1730,H_-0.95_-0.55_-0.00149133.dat,-0.95,-0.55,-0.001491,0,0,val,0.095346,-0.091484,...,-0.062584,0.039609,-0.095202,0.089802,-0.070115,0.085563,-0.002530,0.029463,0.066589,-0.044496
546581,2484,H_-0.5_0.7_1.00063.dat,-0.50,0.70,1.000633,1,1,val,0.095337,0.052835,...,0.014884,0.086080,0.094564,0.043443,-0.009483,-0.083599,-0.095106,-0.048218,0.004051,0.080845


#### Checking train/val/test splits again

In [9]:
%%time
n_total = len(simulation.dataframe)
n_train = len(simulation.dataframe[simulation.dataframe.type_of == "train"])
n_val = len(simulation.dataframe[simulation.dataframe.type_of == "val"])
n_test = len(simulation.dataframe[simulation.dataframe.type_of == "test"])
n_train_hamiltonians = len(simulation.train_ids)
n_val_hamiltonians = len(simulation.val_ids)
n_test_hamiltonians = len(simulation.test_ids)
n_total_hamiltonians = n_train_hamiltonians + n_val_hamiltonians + n_test_hamiltonians

print("% train: ", n_train/n_total)
print("% val: ",  n_val/n_total)
print("% test: ", n_test/n_total)
print("% train + val + test: ", (n_train+n_val+n_test)/n_total)
print("\n")
print("number of train hamiltonians: ", n_train_hamiltonians)
print("number of val hamiltonians: ", n_val_hamiltonians)
print("number of test hamiltonians: ", n_test_hamiltonians)
print("total number of hamiltonians: ", n_total_hamiltonians)
print("\n")
print("train ids: ", simulation.train_ids)
print("val ids: ", simulation.val_ids)
print("test ids: ", simulation.test_ids)

% train:  0.09358329522938577
% val:  0.8422496570644719
% test:  0.06416704770614236
% train + val + test:  1.0


number of train hamiltonians:  614
number of val hamiltonians:  5526
number of test hamiltonians:  421
total number of hamiltonians:  6561


train ids:  [1749, 6342, 2187, 1186, 2065, 4522, 6085, 802, 3811, 6228, 688, 4152, 4257, 2834, 2713, 1011, 2038, 799, 4224, 596, 3561, 2522, 4565, 1921, 1033, 4736, 4184, 719, 5841, 1744, 4854, 1502, 114, 1235, 5363, 4261, 2029, 4343, 4944, 4412, 1024, 4899, 4705, 1012, 1210, 3922, 957, 2364, 4457, 5594, 6170, 1521, 1512, 2811, 392, 2957, 1176, 2253, 6097, 4997, 1565, 5375, 3100, 526, 1132, 4946, 1619, 487, 6353, 4330, 5432, 4806, 4415, 1529, 4466, 4626, 1290, 2596, 3221, 6505, 1839, 3425, 3082, 159, 5154, 2899, 4766, 5797, 959, 2586, 4377, 181, 5898, 2840, 5854, 870, 4885, 5194, 3880, 5372, 4112, 3766, 421, 616, 5320, 3643, 3960, 426, 4325, 6078, 4491, 3329, 6262, 5522, 1956, 13, 9, 178, 4950, 1628, 3274, 4574, 4971, 2490, 385, 6041,

#### Checking summaries

In [10]:
%%time
ham_summary = simulation.hamiltonian_summary
print("length of ham_summary: ", len(ham_summary))
ham_summary

length of ham_summary:  6561
CPU times: user 101 µs, sys: 0 ns, total: 101 µs
Wall time: 60.6 µs


,t1,t2,type_of,0,1,phase,pred_phase
id,,,,,,,
0,-2.0,-2.00,test,0.568182,0.431818,999,0
1,-2.0,-1.95,test,0.750000,0.250000,0,0
2,-2.0,-1.90,test,0.818182,0.181818,0,0
3,-2.0,-1.85,test,0.859091,0.140909,0,0
4,-2.0,-1.80,val,0.918182,0.081818,0,0
...,...,...,...,...,...,...,...
6556,2.0,1.80,val,0.931818,0.068182,0,0
6557,2.0,1.85,test,0.872727,0.127273,0,0
6558,2.0,1.90,test,0.831818,0.168182,0,0


In [11]:
eigen_summary = simulation.eigenvector_summary
print("length of ham_summary: ", len(eigen_summary))
eigen_summary

length of ham_summary:  1443420


,id,phase,pred_phase,type_of
0,0,999,0,test
1,0,999,1,test
2,0,999,1,test
3,0,999,0,test
4,0,999,1,test
...,...,...,...,...
1443415,6560,999,0,test
1443416,6560,999,1,test
1443417,6560,999,0,test
1443418,6560,999,1,test


#### Checking accuracies

In [12]:
simulation.accuracy

{'eigenvector_train': 0.9914495114006515,
 'eigenvector_val': 0.9743666304741223,
 'eigenvector_test': 0.794493006993007,
 'hamiltonian_train': 1.0,
 'hamiltonian_val': 1.0,
 'hamiltonian_test': 1.0}

#### Checking data stored in  memory

In [13]:
ham_summary_list = simulation.hamiltonian_summary_list
ham_summary_list

[]

In [14]:
eigen_summary_list = simulation.eigenvector_summary_list
eigen_summary_list

[]

In [15]:
accuracy_list = simulation.accuracy_list
accuracy_list

{'eigenvector_train': [],
 'eigenvector_val': [],
 'eigenvector_test': [],
 'hamiltonian_train': [],
 'hamiltonian_val': [],
 'hamiltonian_test': []}